# IA - Projeto 2: Fraude Bancária

Para este Projeto foi-nos proposta a análise de modelos de IA treinados por 3 algoritmos de supervised learning à nossa escolha de forma a percebermos como funcionam e o pré-processamento que a análise de dados e de algoritmos de classsificação precisam.

Assim, para o desenvolvimento deste trabalho selecionamos treinar o nosso modelo com os seguintes algoritmos de supervised learning no âmbito dos problemas de classsificação:
- Decision Tree
- Neural Network
- K Nearest Neighbour (KNN)

Ao longo deste Notebook vamos apresentar todos os passos que demos para chegar aos resultados obtidos pelos nossos modelos.

## Dataset

Este dataset que estamos a usar para treinar o nosso algoritmo contém 37 variáveis e 10000 linhas de dados

## Data pre processing

Depois de analisar o dataset verificamos que para treinarmos os modelos, seria necessário fazer um pré-processamento dos dados para os limpar, balancear (uma vez que são desbalanceados, o número de fraudes é mesmo inferior ao número de não fraudes), converter para a mesma escala e transformar os valores das varíaveis categóricas por valores numéricos.
Assim, apresentamos em seguida os passos que demos durante esta fase do Projeto.

### Deal with missing values

Corrijimos os valores em falta, usando este código:

In [12]:
def _remove_missing_values(self):
    columns_to_check = ['prev_address_months_count', 'current_address_months_count', 'intended_balcon_amount',
                        'bank_months_count', 'session_length_in_minutes']
    self.data = self.data[~(self.data[columns_to_check] == -1).any(axis=1)]

### Normalize values from variables

Foi necessário também fazer conversão de valores, uma vez que tínhamos variáveis categóricas que precisamos de transformar em valores numéricos de forma a podermos usar os algoritmos da library sklearn:

In [13]:
def preprocess_data(self):
    numeric_cols = ['prev_address_months_count', 'current_address_months_count', 'customer_age',
                    'days_since_request', 'intended_balcon_amount', 'zip_count_4w', 'velocity_6h', 'velocity_24h',
                    'velocity_4w', 'bank_branch_count_8w', 'date_of_birth_distinct_emails_4w', 'credit_risk_score',
                    'bank_months_count', 'proposed_credit_limit', 'session_length_in_minutes',
                    'device_distinct_emails_8w', 'month']
    string_cols = ['payment_type', 'employment_status', 'housing_status', 'source', 'device_os']
    for strCol in string_cols:
        self.encoder.fit(self.data[strCol])  # Fit the encoder to the data
        self.data[strCol] = self.encoder.transform(self.data[strCol])  # Encode and replace the data


Usamos também uma função de transformação de determinados parâmetros tendo em conta os dados de treino:

In [14]:
 def scale_data(self):
        self.data = pd.DataFrame(self.scaler.fit_transform(self.data), columns=self.data.columns)
        print(self.data)

## Algorithms used to train models

### Decision Tree

Começamos por apresentar os resultados obtidos com o Decision Tree:

In [1]:
class DecisionTreeFraudDetector:
    def __init__(self, data):
        self.data = data
        self.xtrain, self.xtest, self.ytrain, self.ytest = self._split_data()

    def _split_data(self):
        x = self.data.drop('fraud_bool', axis=1)
        y = self.data['fraud_bool']
        X_encoded = pd.get_dummies(x, columns=['payment_type', 'employment_status', 'housing_status', 'source',
                                               'device_os'])

        oversampler = SMOTE(random_state=42)
        X_resampled, y_resampled = oversampler.fit_resample(X_encoded, y)

        xtrain, xtest, ytrain, ytest = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
        return xtrain, xtest, ytrain, ytest

    def train(self):
        self.clf = DecisionTreeClassifier(max_depth=5, min_samples_split=5)
        self.clf.fit(self.xtrain, self.ytrain)

    def predict(self, input_data):
        return self.clf.predict(input_data)

    def evaluate(self):
        y_pred_proba = self.clf.predict_proba(self.xtest)[:, 1]
        fpr, tpr, thresholds = roc_curve(self.ytest, y_pred_proba)
        auc_roc = roc_auc_score(self.ytest, y_pred_proba)

        # Print AUC-ROC score
        print("AUC-ROC Score:", auc_roc)
        for i, threshold in enumerate(thresholds):
            if fpr[i] <= 0.05:
                print("Threshold: {:.2f}, FPR: {:.2f}, TPR: {:.2f}".format(threshold, fpr[i], tpr[i]))

        # Plot ROC curve
        plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc_roc)
        plt.plot([0, 1], [0, 1], 'k--')  # Random guessing line
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC)')
        plt.legend(loc='lower right')
        plt.show()

### Random Forest

Neste modelo usamos o Random Forest para verificarmos se obtíamos melhores resultados do que com a decision tree:

In [2]:
class RandomForest:
    def __init__(self, data):
        self.data = data
        self.X_train, self.X_test, self.y_train, self.y_test = self._split_data()

    def _split_data(self):
        X = self.data.drop('fraud_bool', axis=1)
        y = self.data['fraud_bool']

        # Label encoding
        label_encoder = LabelEncoder()
        X_encoded = X.copy()
        for col in ['payment_type', 'employment_status', 'housing_status', 'source', 'device_os']:
            X_encoded[col] = label_encoder.fit_transform(X[col])

        # Sparse one-hot encoding
        X_encoded = pd.get_dummies(X_encoded, columns=['payment_type', 'employment_status', 'housing_status', 'source', 
                                                       'device_os'], sparse=True)

        # Perform oversampling using SMOTE
        oversampler = SMOTE(random_state=42)
        X_resampled, y_resampled = oversampler.fit_resample(X_encoded, y)

        X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
        return X_train, X_test, y_train, y_test

    def train(self):
        self.clf = RandomForestClassifier(n_jobs=-1)  # Enable parallel processing
        self.clf.fit(self.X_train, self.y_train)

    def predict(self, X_test):
        return self.clf.predict(X_test)

    def evaluate(self):
        y_pred_proba = self.clf.predict_proba(self.X_test)[:, 1]
        fpr, tpr, thresholds = roc_curve(self.y_test, y_pred_proba)
        auc_roc = roc_auc_score(self.y_test, y_pred_proba)

        # Print AUC-ROC score
        print("AUC-ROC Score:", auc_roc)
        for i, threshold in enumerate(thresholds):
            if fpr[i] <= 0.05:
                print("Threshold: {:.2f}, FPR: {:.2f}, TPR: {:.2f}".format(threshold, fpr[i], tpr[i]))

        # Plot ROC curve
        plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc_roc)
        plt.plot([0, 1], [0, 1], 'k--')  # Random guessing line
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC)')
        plt.legend(loc='lower right')
        plt.show()

### Neural Network

De seguida, treinamos o nosso modelo com as Neural Network:

In [3]:
import itertools

from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

#import Sampler
from src.FraudDetectorNN import FraudDetectorNN


def main():
    #sampler = Sampler.Sampler('../datasets/Base.csv')
    #data = sampler.sample(50000, 0.2)
    data = pd.read_csv('../datasets/Base.csv')
    # fraudNN = FraudDetectorNN(data)
    x = data.drop(['fraud_bool', 'bank_months_count', 'prev_address_months_count'], axis=1)
    y = data['fraud_bool']

    for column in ['prev_address_months_count', 'current_address_months_count', 'intended_balcon_amount', 'bank_months_count', 
                   'session_length_in_minutes']:
        values = data[data[column] == -1][column]
        print(column, " : ", len(values))
        print()

    # Preprocessing
    numeric_cols = ['prev_address_months_count', 'current_address_months_count', 'customer_age',
                    'days_since_request', 'intended_balcon_amount', 'zip_count_4w', 'velocity_6h',
                    'velocity_24h', 'velocity_4w', 'bank_branch_count_8w', 'date_of_birth_distinct_emails_4w',
                    'credit_risk_score', 'bank_months_count', 'proposed_credit_limit',
                    'session_length_in_minutes', 'device_distinct_emails_8w', 'month']
    onehot_encoding = ['payment_type', 'employment_status', 'housing_status', 'device_os']
    label_encoding = ['source']
    columns_to_check = ['prev_address_months_count', 'current_address_months_count', 'intended_balcon_amount',
                        'bank_months_count', 'session_length_in_minutes']

    onehot_transformer = ColumnTransformer([('encoder', OneHotEncoder(), onehot_encoding)], remainder='passthrough')
    labelencoder = LabelEncoder()
    scaler = StandardScaler()

    for strCol in label_encoding:
        labelencoder.fit(x[strCol])
        x[strCol] = labelencoder.transform(x[strCol])

    x = pd.DataFrame(x)
    x = pd.DataFrame(onehot_transformer.fit_transform(x))
    x = scaler.fit_transform(x)
    x = pd.DataFrame(scaler.fit_transform(x))

    oversampler = SMOTE(random_state=42)
    x, y = oversampler.fit_resample(x, y)

    # Splitting data into training and testing sets
    xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.75, random_state=0)

    print("Length of x: ", len(xTrain))

    neural_network = MLPClassifier(random_state=0)

    hidden_layers_list = []
    for num_layers in range(1, 3):  # Varying number of hidden layers from 1 to 2
        for layer_size in itertools.product(range(20, 101, 10), repeat=num_layers):
            hidden_layers_list.append(layer_size)

    print("Length: ", len(hidden_layers_list))
    print(hidden_layers_list)

    param_distributions = {'hidden_layer_sizes': hidden_layers_list,
                           'activation': ['relu', 'logistic'],
                           'solver': ['adam'],
                           }

    grid_search = RandomizedSearchCV(neural_network, param_distributions, cv=3, n_iter=20, scoring="roc_auc", verbose=3, 
                                     n_jobs=-1)
    grid_search.fit(xTrain, yTrain)

    print("Best Parameters:")
    print(grid_search.best_params_)
    print()

    best_neural_network = grid_search.best_estimator_
    best_neural_network.fit(xTrain, yTrain)

    # Predict probabilities for the testing set
    y_pred_proba = best_neural_network.predict_proba(xTest)[:, 1]

    # Calculate ROC curve and AUC-ROC score
    fpr, tpr, thresholds = roc_curve(yTest, y_pred_proba)
    auc_roc = roc_auc_score(yTest, y_pred_proba)

    # Print AUC-ROC score
    print("AUC-ROC Score:", auc_roc)

    # Print threshold, FPR, and TPR for fpr <= 0.05
    for i, threshold in enumerate(thresholds):
        if fpr[i] <= 0.05:
            print("Threshold: {:.2f}, FPR: {:.2f}, TPR: {:.2f}".format(threshold, fpr[i], tpr[i]))

    # Plot ROC curve
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc_roc)
    plt.plot([0, 1], [0, 1], 'k--')  # Random guessing line
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc='lower right')
    plt.show()

if __name__ == '__main__':
    main()

ModuleNotFoundError: No module named 'src'

### KNN

Por último, utilizamos o KNN para treinar o nosso modelo:

In [17]:
class KNNFraudDetector:
    def __init__(self, data):
        self.data = data
        self.xtrain, self.xtest, self.ytrain, self.ytest = self._split_data()

    def _split_data(self):
        x = self.data.drop('fraud_bool', axis=1)
        y = self.data['fraud_bool']
        X_encoded = pd.get_dummies(x, columns=['payment_type', 'employment_status', 'housing_status', 'source',
                                               'device_os'])

        oversampler = SMOTE(random_state=42)
        X_resampled, y_resampled = oversampler.fit_resample(X_encoded, y)

        xtrain, xtest, ytrain, ytest = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
        return xtrain, xtest, ytrain, ytest

    def train(self):
        self.clf = KNeighborsClassifier(n_neighbors=5)
        self.clf.fit(self.xtrain, self.ytrain)

    def predict(self, input_data):
        return self.clf.predict(input_data)

    def evaluate(self):
        y_pred_proba = self.clf.predict_proba(self.xtest)[:, 1]
        fpr, tpr, thresholds = roc_curve(self.ytest, y_pred_proba)
        auc_roc = roc_auc_score(self.ytest, y_pred_proba)

        # Print AUC-ROC score
        print("AUC-ROC Score:", auc_roc)
        for i, threshold in enumerate(thresholds):
            if fpr[i] <= 0.05:
                print("Threshold: {:.2f}, FPR: {:.2f}, TPR: {:.2f}".format(threshold, fpr[i], tpr[i]))

        # Plot ROC curve
        plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc_roc)
        plt.plot([0, 1], [0, 1], 'k--')  # Random guessing line
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC)')
        plt.legend(loc='lower right')
        plt.show()

Com este modelo conseguimos obter um resultado ROC de **91%**. Um dos parâmetros do algoritmo que usamos `KNeighborsClassifier` que tem bastante impacto neste resultado é no `n_neighbors` (que representa o número de vizinhos do nó atual que vamos visitar). Reparamos que quanto maior o número de vizinhos, pior o desempenho deste modelo. Pelo que o resultado ótimo, foi encontrado com `n_neighbors=5`.

Durante o período de testes e experiência, reparamos também que a normalização das variáveis tem um grande impacto neste algoritmo, uma vez que quando usamos outra forma de pre-processamento obtivemos um resultado de 60%. Muito possivelmente, porque os relevantes para o modelo não estavam tratados.

## Result analysis

Para conseguirmos analisar e comparar os 3 modelos que treinamos, usamos várias métricas de entre as quais destacamos:
- **AUC-ROC**
- **Confusion Matrix**
- **Learning curve**

## Comparison between Models

## Conclusion